In [6]:
import pandas as pd
import numpy as np
import pybedtools
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

In [4]:
# merge overlapped gene body
!bedtools merge -i gene.sorted.bed > non_overlap_intragenic.bed
# get intergenic and intragenic region
!bedtools subtract -a mm10.whole_genome.bed -b non_overlap_intragenic.bed > non_overlap_intergenic.bed
!bedtools subtract -a non_overlap_intergenic.bed -b mm10-blacklist.v2.bed > non_overlap_intergenic.subtract_black.bed
!bedtools subtract -a non_overlap_intragenic.bed -b mm10-blacklist.v2.bed > non_overlap_intragenic.subtract_black.bed

# subdivide dmr into intra- and intergenic
!bedtools intersect -a ../DG_DMR.decrease.mch_group.bed -b non_overlap_intergenic.subtract_black.bed -wa -u > DG_DMR.decrease.intergenic.bed
!bedtools intersect -a ../DG_DMR.decrease.mch_group.bed -b non_overlap_intragenic.subtract_black.bed -wa -u > DG_DMR.decrease.intragenic.bed
!bedtools intersect -a ../DG_DMR.increase.mch_group.bed -b non_overlap_intragenic.subtract_black.bed -wa -u > DG_DMR.increase.intragenic.bed
!bedtools intersect -a ../DG_DMR.increase.mch_group.bed -b non_overlap_intergenic.subtract_black.bed -wa -u > DG_DMR.increase.intergenic.bed

# get fasta of each region
!bedtools getfasta -fi /home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa -bed non_overlap_intergenic.subtract_black.bed > intergenic.fa
!bedtools getfasta -fi /home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa -bed non_overlap_intergenic.subtract_black.bed > intergenic.fa



In [7]:
dmr_data = pd.read_csv('../axis-mch_rms_results_collapsed_passfilter_annotated.csv', index_col=0)
dmr_data.index = dmr_data.index.map(lambda i: f'DGmCH_{i}')
dmr_data.index.name = 'DMR_id'
dmr_data.head()

,#chr,start,end,number_of_dms,low,midlow,midhigh,high,rho,trend,...,annot-stop_codon.all,annot-TSS.all,annot-TSS.protein_coding,annot-UTR3.all,annot-UTR3.protein_coding,annot-UTR5.all,annot-UTR5.protein_coding,annot-CGI,annot-CGI_shore,annot-Transposon
DMR_id,,,,,,,,,,,,,,,,,,,,,
DGmCH_0,chr1,3026310,3026310,1,0.396226,0.309091,0.631579,0.627907,0.801534,increasing,...,False,False,False,False,False,False,False,False,False,True
DGmCH_1,chr1,3069892,3069892,1,0.807692,0.583333,0.743590,0.960000,0.510749,no trend,...,False,False,False,False,False,False,False,False,False,True
DGmCH_2,chr1,3084771,3084771,1,0.413793,0.387097,0.645161,0.766667,0.926190,increasing,...,False,False,False,False,False,False,False,False,False,True
DGmCH_3,chr1,3092179,3092179,1,0.744681,0.666667,0.921875,0.944444,0.814380,increasing,...,False,False,False,False,False,False,False,False,False,True
DGmCH_4,chr1,3118806,3118806,1,0.697674,0.933333,0.868421,1.000000,0.837758,increasing,...,False,False,False,False,False,False,False,False,False,True


In [8]:
inc_intra_bed = pd.read_csv('DG_DMR.increase.intragenic.bed', 
                            header=None, sep='\t', index_col=3, 
                            names=['chrom', 'start', 'end', 'DMR_id'])
inc_intra_total_dms = dmr_data.loc[inc_intra_bed.index]['number_of_dms'].sum()
inc_inter_bed = pd.read_csv('DG_DMR.increase.intergenic.bed', 
                            header=None, sep='\t', index_col=3, 
                            names=['chrom', 'start', 'end', 'DMR_id'])
inc_inter_total_dms = dmr_data.loc[inc_inter_bed.index]['number_of_dms'].sum()
dec_intra_bed = pd.read_csv('DG_DMR.decrease.intragenic.bed', 
                            header=None, sep='\t', index_col=3, 
                            names=['chrom', 'start', 'end', 'DMR_id'])
dec_intra_total_dms = dmr_data.loc[dec_intra_bed.index]['number_of_dms'].sum()
dec_inter_bed = pd.read_csv('DG_DMR.decrease.intergenic.bed', 
                            header=None, sep='\t', index_col=3, 
                            names=['chrom', 'start', 'end', 'DMR_id'])
dec_inter_total_dms = dmr_data.loc[dec_inter_bed.index]['number_of_dms'].sum()

## Gene meta

In [11]:
gene_meta = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
                        sep='\t', index_col='gene_id')
gene_meta['length'] = gene_meta['end'] - gene_meta['start']
gene_name_to_id = {v: k for k, v in gene_meta['gene_name'].items()}
gene_meta.head()

,chrom,source,feature,start,end,score,strand,phase,transcript_id,gene_type,...,transcript_type,transcript_status,transcript_name,exon_number,exon_id,level,mgi_id,havana_gene,tag,length
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000102693.1,chr1,HAVANA,gene,3073253,3074322,.,+,.,NaN,TEC,...,NaN,NaN,NaN,NaN,NaN,2,MGI:1918292,OTTMUSG00000049935.1,NaN,1069
ENSMUSG00000064842.1,chr1,ENSEMBL,gene,3102016,3102125,.,+,.,NaN,snRNA,...,NaN,NaN,NaN,NaN,NaN,3,MGI:5455983,NaN,NaN,109
ENSMUSG00000051951.5,chr1,HAVANA,gene,3205901,3671498,.,-,.,NaN,protein_coding,...,NaN,NaN,NaN,NaN,NaN,2,MGI:3528744,OTTMUSG00000026353.2,NaN,465597
ENSMUSG00000102851.1,chr1,HAVANA,gene,3252757,3253236,.,+,.,NaN,processed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,1,MGI:5011141,OTTMUSG00000049958.1,pseudo_consens,479
ENSMUSG00000103377.1,chr1,HAVANA,gene,3365731,3368549,.,-,.,NaN,TEC,...,NaN,NaN,NaN,NaN,NaN,2,MGI:5610408,OTTMUSG00000049960.1,NaN,2818


## Intra Inter genic

In [12]:
intragenic_df = pd.read_csv(
    'non_overlap_intragenic.subtract_black.gene_id_anno.bed',
    header=None,
    sep='\t',
    index_col=None,
    names=['chrom', 'start', 'end', 'gene_ids'])
intragenic_df.index = intragenic_df.chrom + ':' + intragenic_df.start.astype(str) + '-' + intragenic_df.end.astype(str)

intergenic_df = pd.read_csv(
    'non_overlap_intergenic.subtract_black.bed',
    header=None,
    sep='\t',
    index_col=None,
    names=['chrom', 'start', 'end'])
intergenic_df.index = intergenic_df.chrom + ':' + intergenic_df.start.astype(str) + '-' + intergenic_df.end.astype(str)


## Background C

In [13]:
def count_c(file_path):
    records = {}
    with open(file_path) as f:
        name = None
        for line in f:
            if line.startswith('>'):
                name = line[1:-1]
            else:
                records[name] = line.lower().count('c')
    return pd.Series(records)

In [14]:
intragenic_c = count_c('intragenic.fa')
intergenic_c = count_c('intergenic.fa')

total_intragenic_c = intragenic_c.sum()
total_intergenic_c = intergenic_c.sum()

In [15]:
total_intragenic_c, total_intergenic_c, inc_intra_total_dms, dec_intra_total_dms

(253112183, 261977002, 114665, 71795)

## Per gene DMS

In [16]:
dms_dict = dmr_data['number_of_dms'].to_dict()

def get_dms(value):
    if isinstance(value, float):
        return 0
    else:
        return sum([dms_dict.get(dmr_id, 0) for dmr_id in value.split(',')])

## Significance cutoff

In [20]:
p_cutoff = 0.01
odds_cutoff = 1.5

## Test Decreasing DMR

In [22]:
intra_dec_df = pd.read_csv('non_overlap_intragenic.subtract_black.dec_dmr.bed', 
                           sep='\t', header=None, na_values='.', names=['chrom', 'start', 'end', 'DMR_IDs'])
intra_dec_df['dms_in'] = intra_dec_df['DMR_IDs'].apply(get_dms)

intra_dec_df.index = intra_dec_df.chrom + ':' + intra_dec_df.start.astype(str) + '-' + intra_dec_df.end.astype(str)
intra_dec_df['other_in'] = intragenic_c - intra_dec_df['dms_in']

intra_dec_df['dms_out'] = dec_intra_total_dms - intra_dec_df['dms_in']
intra_dec_df['other_out'] = total_intragenic_c - intragenic_c - intra_dec_df['dms_out']

intra_dec_df.tail()

,chrom,start,end,DMR_IDs,dms_in,other_in,dms_out,other_out
chrY:10510481-10510859,chrY,10510481,10510859,NaN,0,91,71795,253040297
chrY:10533608-10536041,chrY,10533608,10536041,NaN,0,520,71795,253039868
chrY:10553452-10553991,chrY,10553452,10553991,NaN,0,153,71795,253040235
chrY:10616360-10616902,chrY,10616360,10616902,NaN,0,153,71795,253040235
chrY:10626742-10629109,chrY,10626742,10629109,NaN,0,551,71795,253039837


In [24]:
def test_enrich(row):
    return pd.Series(
        fisher_exact([[row['dms_in'], row['dms_out']],
                      [row['other_in'], row['other_out']]], 'greater'))

test_result = intra_dec_df.apply(test_enrich, axis=1)
test_result.columns = ['odds', 'p']
test_result = test_result[test_result['odds'] > 1].copy()

judge, adj_p, _, _ = multipletests(test_result['p'], method='fdr_bh')

test_result['judge'] = judge
test_result['adj_p'] = adj_p
test_result['-lgp'] = -np.log10(adj_p)

In [25]:
sig_dec_intra_result = test_result[(test_result['adj_p'] < p_cutoff) & 
                                   (test_result['odds'] > odds_cutoff)].index
total_dec_intra_result = pd.concat([
    intra_dec_df.loc[sig_dec_intra_result],
    test_result.loc[sig_dec_intra_result],
    intragenic_df.loc[sig_dec_intra_result, ['gene_ids']]
],
                                   axis=1,
                                   sort=True)

total_dec_intra_result.to_csv('FINAL.Intragenic.decrease.results.csv')

In [26]:
nofilter_dec_intra_result = pd.concat([
    intra_dec_df.loc[test_result.index],
    test_result,
    intragenic_df.loc[test_result.index, ['gene_ids']]
],
                                   axis=1,
                                   sort=True)
nofilter_dec_intra_result.to_csv('FINAL.nofilter.Intragenic.decrease.results.csv')

In [27]:
judges = (total_dec_intra_result['dms_in'] > 50) & \
         (total_dec_intra_result['odds'] > 2.5)

length_cutoff = 10000

dec_genes = set()
for gene_ids in total_dec_intra_result[judges]['gene_ids']:
    for gene in gene_ids.split(','):
        gene_length = gene_meta.at[gene, 'length']
        if gene_length < length_cutoff:
            continue
        dec_genes.add(gene)
with open('dec_genes.txt', 'w') as f:
    for gene in dec_genes:
        f.write(gene.split('.')[0]+'\n')
# this number decludes many small gene 
len(dec_genes)

183

## Test increasing DMR

In [28]:
intra_inc_df = pd.read_csv('non_overlap_intragenic.subtract_black.inc_dmr.bed', 
                           sep='\t', header=None, na_values='.', names=['chrom', 'start', 'end', 'DMR_IDs'])
intra_inc_df['dms_in'] = intra_inc_df['DMR_IDs'].apply(get_dms)

intra_inc_df.index = intra_inc_df.chrom + ':' + intra_inc_df.start.astype(str) + '-' + intra_inc_df.end.astype(str)
intra_inc_df['other_in'] = intragenic_c - intra_inc_df['dms_in']

intra_inc_df['dms_out'] = inc_intra_total_dms - intra_inc_df['dms_in']
intra_inc_df['other_out'] = total_intragenic_c - intragenic_c - intra_inc_df['dms_out']

intra_inc_df.head()

,chrom,start,end,DMR_IDs,dms_in,other_in,dms_out,other_out
chr1:3073252-3074321,chr1,3073252,3074321,NaN,0,160,114665,252997358
chr1:3102015-3102124,chr1,3102015,3102124,NaN,0,20,114665,252997498
chr1:3205900-3671497,chr1,3205900,3671497,"DGmCH_8,DGmCH_22",2,89686,114663,252907832
chr1:3680154-3681787,chr1,3680154,3681787,NaN,0,336,114665,252997182
chr1:3752009-3754359,chr1,3752009,3754359,NaN,0,474,114665,252997044


In [29]:
def test_enrich(row):
    return pd.Series(
        fisher_exact([[row['dms_in'], row['dms_out']],
                      [row['other_in'], row['other_out']]], 'greater'))

test_result = intra_inc_df.apply(test_enrich, axis=1)
test_result.columns = ['odds', 'p']
test_result = test_result[test_result['odds'] > 1].copy()

judge, adj_p, _, _ = multipletests(test_result['p'], method='fdr_bh')

test_result['judge'] = judge
test_result['adj_p'] = adj_p
test_result['-lgp'] = -np.log10(adj_p)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log10
  


In [30]:
sig_inc_intra_result = test_result[(test_result['adj_p'] < p_cutoff) & 
                                   (test_result['odds'] > odds_cutoff)].index
total_inc_intra_result = pd.concat([
    intra_inc_df.loc[sig_inc_intra_result],
    test_result.loc[sig_inc_intra_result],
    intragenic_df.loc[sig_inc_intra_result, ['gene_ids']]
],
                                   axis=1,
                                   sort=True)

total_inc_intra_result.to_csv('FINAL.Intragenic.increase.results.csv')

In [31]:
nofilter_inc_intra_result = pd.concat([
    intra_inc_df.loc[test_result.index],
    test_result,
    intragenic_df.loc[test_result.index, ['gene_ids']]
],
                                   axis=1,
                                   sort=True)
nofilter_inc_intra_result.to_csv('FINAL.nofilter.Intragenic.increase.results.csv')

In [32]:
judges = (total_inc_intra_result['dms_in'] > 50) & \
         (total_inc_intra_result['odds'] > 2.5)

length_cutoff = 10000

inc_genes = set()
for gene_ids in total_inc_intra_result[judges]['gene_ids']:
    for gene in gene_ids.split(','):
        gene_length = gene_meta.at[gene, 'length']
        if gene_length < length_cutoff:
            continue
        inc_genes.add(gene)
with open('inc_genes.txt', 'w') as f:
    for gene in inc_genes:
        f.write(gene.split('.')[0]+'\n')
# this number includes many small gene 
len(inc_genes)

245